In [1]:
# Ejercicio 1: Introducción a los principales algoritmos de RL
#Define el entorno del juego
class Environment:
    def __init__(self):
        self.state_space = [0, 1, 2, 3] # Estados posibles
        self.action_space = [0, 1] # Acciones posibles 
        self.rewards = {0:-1, 1: -1, 2: -1, 3: 10} # Recompensas por estado

In [2]:
#Crea una instancia del entorno
env = Environment()

In [3]:
# Muestra información del entorno
print("Estados:", env.state_space)
print("Acciones:", env.action_space)
print("Recompensas:", env.rewards)

Estados: [0, 1, 2, 3]
Acciones: [0, 1]
Recompensas: {0: -1, 1: -1, 2: -1, 3: 10}


In [4]:
 #Ejercicio 2: Q-Learning
import numpy as np

In [5]:
# Inicializa la tabla Q con valores arbitrarios
Q = np.zeros((len(env.state_space), len(env.action_space)))

In [6]:
# Define los parámetros del algoritmo
alpha = 0.1 # Tasa de aprendizaje
gamma = 0.9 # Factor de descuento

In [7]:
#Entrena el agente utilizando Q-Learning
for _ in range(1000):
    state = np.random.choice(env.state_space)  # Estado inicial aleatorio
    
    while state != 3:  # Hasta llegar al estado objetivo
        action = np.random.choice(env.action_space)  # Selecciona una acción aleatoria
        next_state = state + action  # Determina el siguiente estado
        reward = env.rewards[next_state]  # Obtiene la recompensa del siguiente estado
        
        # Actualiza la tabla Q
        Q[state, action] = Q[state, action] + alpha * (reward + gamma * np.max(Q[next_state]) - Q[state, action])
        
        state = next_state  # Actualiza el estado actual

In [8]:
#Muestra la función Q-valor aprendida
print("Función Q-Valor aprendida :")
print(Q)

Función Q-Valor aprendida :
[[ 4.58  6.2 ]
 [ 6.2   8.  ]
 [ 8.   10.  ]
 [ 0.    0.  ]]


In [9]:
#Ejercicio 3: Sarsa
#Reinicializa la tabla Q con valores arbitrarios
Q = np.zeros((len(env.state_space), len(env.action_space)))

In [10]:
# Entrena el agente utilizando Sarsa
for _ in range(1000):
    state = np.random.choice(env.state_space) # Estado inicial aleatorio
    action = np.random.choice (env.action_space) # Selecciona una acción aleatoria
    while state != 3: # Hasta llegar al estado objetivo
        next_state = state + action
        next_action = np.random.choice (env.action_space) # Selecciona una acción aleatorial
        reward = env.rewards [next_state]
        Q[state, action] = Q[state, action] + alpha * (reward + gamma * Q[next_state, next_action] - Q[state, action])
        state = next_state
        action = next_action

In [11]:
# Muestra la función Q-valor aprendida con Sarsa
print("Función Q-Valor aprendida con Sarsa:")
print(Q)

Función Q-Valor aprendida con Sarsa:
[[ 0.72631831  3.47159331]
 [ 3.6083861   6.81067443]
 [ 6.16207759 10.        ]
 [ 0.          0.        ]]


In [12]:
# Ejercicio 4: Política de Gradiente de Montecarlo
# Inicializa la política con probabilidades uniformes
policy= np.ones((len(env.state_space), len (env.action_space))) / len(env.action_space)


In [13]:
print(policy)

[[0.5 0.5]
 [0.5 0.5]
 [0.5 0.5]
 [0.5 0.5]]


In [14]:
# Define la función de recompensa promedio
def average_reward(Q):
    return np.mean([Q[state, np.argmax(policy [state])] for state in env.state_space])
    


In [15]:
# los valores de policy[state] deben sumar exactamente 1. Si no es así, hhay que normalizar las probabilidades:
policy[state] = policy[state] / np.sum(policy[state])
print(policy[state])

[0.5 0.5]


In [16]:
# Entrena la política utilizando Gradiente de Montecarlo
for _ in range(1000):
    state = np.random.choice(env.state_space)  # Estado inicial aleatorio
    while state != 3:  # Hasta llegar al estado objetivo
        
        # Evitar probabilidades negativas y normalizar la política antes de elegir acción
        policy[state] = np.maximum(policy[state], 0)  # Evita probabilidades negativas
        policy[state] = policy[state] / np.sum(policy[state])  # Asegura que sumen 1
        
        # Verificar que las probabilidades sean válidas
        if np.any(policy[state] < 0):
            raise ValueError(f"Las probabilidades contienen valores negativos en el estado {state}: {policy[state]}")
        if not np.isclose(np.sum(policy[state]), 1):
            raise ValueError(f"Las probabilidades no suman 1 en el estado {state}: {policy[state]}")

        action = np.random.choice(env.action_space, p=policy[state])  # Selecciona una acción con la política actual
        next_state = state + action
        reward = env.rewards[next_state]
        
        gradient = np.zeros_like(policy[state])
        gradient[action] = 1
        alpha = 0.01  # Tasa de aprendizaje
        
        # Actualiza la política
        policy[state] += alpha * gradient * (reward - average_reward(Q))
        
        # Evitar probabilidades negativas y normalizar después de la actualización
        policy[state] = np.maximum(policy[state], 0)
        policy[state] = policy[state] / np.sum(policy[state])  # Normaliza la política nuevamente

        # Verificar nuevamente después de la actualización
        if np.any(policy[state] < 0):
            raise ValueError(f"Las probabilidades contienen valores negativos en el estado {state}: {policy[state]}")
        if not np.isclose(np.sum(policy[state]), 1):
            raise ValueError(f"Las probabilidades no suman 1 en el estado {state}: {policy[state]}")
        
        state = next_state

In [17]:

#Muestra la política aprendida
print("Politica aprendida con Gradiente de Montecarlo:")
print(policy)

Politica aprendida con Gradiente de Montecarlo:
[[0.00000000e+00 1.00000000e+00]
 [0.00000000e+00 1.00000000e+00]
 [2.73068119e-17 1.00000000e+00]
 [5.00000000e-01 5.00000000e-01]]
